In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("tip_count_sql").getOrCreate()

22/12/25 19:11:17 WARN Utils: Your hostname, PSui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.7 instead (on interface en0)
22/12/25 19:11:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/25 19:11:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/25 19:11:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# row 데이터 생성
1. wget -i https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2020-03.parquet
    1. (전체 데이터가 필요한 경우) ./install_row_data.sh
2. !conda install pyarrow fastparquet -y
3. convert parquet to csv

In [34]:
!conda install pyarrow fastparquet -y

done
Solving environment: done

## Package Plan ##

  environment location: /usr/local/anaconda3

  added / updated specs:
    - fastparquet
    - pyarrow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    abseil-cpp-20211102.0      |       he9d5cce_0         933 KB
    arrow-cpp-8.0.0            |   py39had1886b_0         7.0 MB
    aws-c-common-0.4.57        |       hb1e8313_1         140 KB
    aws-c-event-stream-0.1.6   |       h23ab428_5          21 KB
    aws-checksums-0.1.9        |       hb1e8313_0          47 KB
    aws-sdk-cpp-1.8.185        |       he271ece_0         1.6 MB
    boost-cpp-1.73.0           |      hca72f7f_12          17 KB
    conda-22.11.1              |   py39hecd8cb5_4         941 KB
    fastparquet-0.5.0          |   py39h67323c0_2         173 KB
    gflags-2.2.2               |       h0a44026_0         104 KB
    glog-0.5.0                 |       h23ab428_0

In [49]:
import pandas as pd
# !pip install pyarrow fastparquet

target_directory = '../../../../data'
target_data = 'fhvhv_tripdata_2020-03'
df = pd.read_parquet('%s/%s.parquet' % (target_directory, target_data))

In [53]:
df.to_csv(f'{target_directory}/{target_data}.csv')

In [56]:
data = spark.read.csv(f'{target_directory}/{target_data}.csv', header=True)

`header=True`

아래와 같은 형식 생성을 방지할 수 있음.
+----+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
| _c0|              _c1|                 _c2|                 _c3|                _c4|                _c5|                _c6|                _c7|         _c8|         _c9|      _c10|     _c11|               _c12| _c13|_c14|     _c15|                _c16|       _c17|_c18|      _c19|               _c20|             _c21|              _c22|            _c23|          _c24|
+----+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|null|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
|   0|           HV0005|              B02510|                null|2020-03-01 00:00:12|               null|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|     8.655|     1199|              24.45|  0.0|0.54|      1.9|                 0.0|       null| 0.0|     19.65|                  N|                N|                 N|               N|             N|


In [57]:
data.show()

22/12/25 21:08:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , hvfhs_license_num, dispatching_base_num, originating_base_num, request_datetime, on_scene_datetime, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, trip_miles, trip_time, base_passenger_fare, tolls, bcf, sales_tax, congestion_surcharge, airport_fee, tips, driver_pay, shared_request_flag, shared_match_flag, access_a_ride_flag, wav_request_flag, wav_match_flag
 Schema: _c0, hvfhs_license_num, dispatching_base_num, originating_base_num, request_datetime, on_scene_datetime, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, trip_miles, trip_time, base_passenger_fare, tolls, bcf, sales_tax, congestion_surcharge, airport_fee, tips, driver_pay, shared_request_flag, shared_match_flag, access_a_ride_flag, wav_request_flag, wav_match_flag
Expected: _c0 but found: 
CSV file: file:///Users/auto/github/spark/data/fhvhv_tripdata_2020-03.csv
+---+-----------------+----------------

In [58]:
data.createOrReplaceTempView("mobility_data")

In [59]:
spark.sql("select * from mobility_data").limit(5).show()

22/12/25 21:09:01 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , hvfhs_license_num, dispatching_base_num, originating_base_num, request_datetime, on_scene_datetime, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, trip_miles, trip_time, base_passenger_fare, tolls, bcf, sales_tax, congestion_surcharge, airport_fee, tips, driver_pay, shared_request_flag, shared_match_flag, access_a_ride_flag, wav_request_flag, wav_match_flag
 Schema: _c0, hvfhs_license_num, dispatching_base_num, originating_base_num, request_datetime, on_scene_datetime, pickup_datetime, dropoff_datetime, PULocationID, DOLocationID, trip_miles, trip_time, base_passenger_fare, tolls, bcf, sales_tax, congestion_surcharge, airport_fee, tips, driver_pay, shared_request_flag, shared_match_flag, access_a_ride_flag, wav_request_flag, wav_match_flag
Expected: _c0 but found: 
CSV file: file:///Users/auto/github/spark/data/fhvhv_tripdata_2020-03.csv
+---+-----------------+----------------

In [73]:
# date time을 날짜와 시간으로 분리
spark.sql('select pickup_date, count(*) as trips '
          'from (select split(pickup_datetime, " ")[0] as pickup_date '
                'from mobility_data ) '
                'group by pickup_date').show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-02|648990|
| 2020-03-01|784260|
| 2020-03-03|697880|
| 2020-03-04|707879|
| 2020-03-05|731165|
| 2020-03-06|872012|
| 2020-03-07|886071|
| 2020-03-08|731222|
| 2020-03-09|628940|
| 2020-03-10|626474|
| 2020-03-11|628601|
| 2020-03-12|643257|
| 2020-03-13|660914|
| 2020-03-15|448125|
| 2020-03-14|569397|
| 2020-03-16|391518|
| 2020-03-17|312298|
| 2020-03-20|261900|
| 2020-03-19|252773|
| 2020-03-18|269233|
+-----------+------+
only showing top 20 rows

